In [32]:
from convokit import Corpus, download
from convokit import text_processing

In [33]:
corpus = Corpus(filename=download("movie-corpus"))
corpus.print_summary_stats()

Number of Speakers: 9035
Number of Utterances: 304713
Number of Conversations: 83097


In [67]:
from cleantext import clean
clean_str = lambda s: clean(
    s,
    fix_unicode=True,  # fix various unicode errors
    to_ascii=True,  # transliterate to closest ASCII representation
    lower=True,  # lowercase text
    no_line_breaks=True,  # fully strip line breaks as opposed to only normalizing them
    no_urls=True,  # replace all URLs with a special token
    no_emails=True,  # replace all email addresses with a special token
    no_phone_numbers=True,  # replace all phone numbers with a special token
    no_numbers=True,  # replace all numbers with a special token
    no_digits=False,  # replace all digits with a special token
    no_currency_symbols=True,  # replace all currency symbols with a special token
    no_punct=True,  # fully remove punctuation
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_phone_number="<PHONE>",
    replace_with_number="<NUMBER>",
    replace_with_digit="0",
    replace_with_currency_symbol="<CUR>",
    lang="en",
)


In [73]:
def text_preprocess(corpus, clean_str):

    # Preprocessing step
    tc = text_processing.textCleaner.TextCleaner(clean_str)
    corpus = tc.transform(corpus)
    sentences = []
    uids = corpus.get_utterance_ids()

    for uid in uids:
        parsed_data = corpus.get_utterance(uid).text
        [sentences.append(i) for i in parsed_data.split()]

    return(sentences)
    
sentences = text_preprocess(corpus, clean_str)

100/304713 utterances processed
200/304713 utterances processed
300/304713 utterances processed
400/304713 utterances processed
500/304713 utterances processed
600/304713 utterances processed
700/304713 utterances processed
800/304713 utterances processed
900/304713 utterances processed
1000/304713 utterances processed
1100/304713 utterances processed
1200/304713 utterances processed
1300/304713 utterances processed
1400/304713 utterances processed
1500/304713 utterances processed
1600/304713 utterances processed
1700/304713 utterances processed
1800/304713 utterances processed
1900/304713 utterances processed
2000/304713 utterances processed
2100/304713 utterances processed
2200/304713 utterances processed
2300/304713 utterances processed
2400/304713 utterances processed
2500/304713 utterances processed
2600/304713 utterances processed
2700/304713 utterances processed
2800/304713 utterances processed
2900/304713 utterances processed
3000/304713 utterances processed
3100/304713 utteran

In [74]:
sentences

['they',
 'do',
 'not',
 'they',
 'do',
 'to',
 'i',
 'hope',
 'so',
 'she',
 'okay',
 'lets',
 'go',
 'wow',
 'okay',
 'youre',
 'gonna',
 'need',
 'to',
 'learn',
 'how',
 'to',
 'lie',
 'no',
 'im',
 'kidding',
 'you',
 'know',
 'how',
 'sometimes',
 'you',
 'just',
 'become',
 'this',
 'persona',
 'and',
 'you',
 'dont',
 'know',
 'how',
 'to',
 'quit',
 'like',
 'my',
 'fear',
 'of',
 'wearing',
 'pastels',
 'the',
 'real',
 'you',
 'what',
 'good',
 'stuff',
 'i',
 'figured',
 'youd',
 'get',
 'to',
 'the',
 'good',
 'stuff',
 'eventually',
 'thank',
 'god',
 'if',
 'i',
 'had',
 'to',
 'hear',
 'one',
 'more',
 'story',
 'about',
 'your',
 'coiffure',
 'me',
 'this',
 'endless',
 'blonde',
 'babble',
 'im',
 'like',
 'boring',
 'myself',
 'what',
 'crap',
 'do',
 'you',
 'listen',
 'to',
 'this',
 'crap',
 'no',
 'then',
 'guillermo',
 'says',
 'if',
 'you',
 'go',
 'any',
 'lighter',
 'youre',
 'gonna',
 'look',
 'like',
 'an',
 'extra',
 'on',
 '<number>',
 'you',
 'always',
 

In [45]:
corpus.get_utterance_ids()

['L1045',
 'L1044',
 'L985',
 'L984',
 'L925',
 'L924',
 'L872',
 'L871',
 'L870',
 'L869',
 'L868',
 'L867',
 'L866',
 'L865',
 'L864',
 'L863',
 'L862',
 'L861',
 'L860',
 'L699',
 'L698',
 'L697',
 'L696',
 'L695',
 'L694',
 'L693',
 'L663',
 'L662',
 'L578',
 'L577',
 'L576',
 'L575',
 'L407',
 'L406',
 'L405',
 'L404',
 'L403',
 'L402',
 'L401',
 'L368',
 'L367',
 'L366',
 'L365',
 'L364',
 'L363',
 'L281',
 'L280',
 'L277',
 'L276',
 'L275',
 'L274',
 'L273',
 'L272',
 'L271',
 'L208',
 'L207',
 'L206',
 'L205',
 'L204',
 'L203',
 'L202',
 'L201',
 'L200',
 'L199',
 'L198',
 'L197',
 'L196',
 'L195',
 'L194',
 'L953',
 'L952',
 'L660',
 'L659',
 'L600',
 'L599',
 'L598',
 'L597',
 'L596',
 'L595',
 'L580',
 'L579',
 'L573',
 'L572',
 'L571',
 'L51',
 'L50',
 'L49',
 'L760',
 'L759',
 'L758',
 'L757',
 'L756',
 'L593',
 'L592',
 'L591',
 'L590',
 'L589',
 'L397',
 'L396',
 'L395',
 'L394',
 'L1052',
 'L1051',
 'L1022',
 'L1021',
 'L1011',
 'L1010',
 'L1009',
 'L1008',
 'L1007',
 '

In [ ]:
from speechbrain.inference.text import GraphemeToPhoneme
# Convert graphemes to phonemes
g2p = GraphemeToPhoneme.from_hparams("speechbrain/soundchoice-g2p", 
                                     run_opts={"device":"cuda"})
